In [73]:
import pandas as pd
import os
import numpy as np
import openpyxl
import json

def LCS(name1, name2):
    # construct a C array
    C = []
    for i in range(1, len(name1) + 2):
        temp = []
        for j in range(1, len(name2) + 2):
            temp.append(0)
        C.append(temp)

    for i in range(1, len(name1) + 1):
        for j in range(1, len(name2) + 1):
            if(name1[i - 1] == name2[j - 1]):
                C[i][j] = C[i - 1][j - 1] + 1
            else:
                C[i][j] = max(C[i][j - 1], C[i - 1][j])

    return C[len(name1)][len(name2)]


def classify(deptName, depts):
    mostLikely = ["",0]
    departments = []
    for i in range(len(depts)):
        departments.append([depts[i], 0])

    for i in range(len(departments)):
        result = LCS(deptName, departments[i][0])
        # print(result)
        if(abs(len(departments[i][0]) - len(deptName)) < 1):
            # print(departments[i])
            # print("plus 1")
            result += 1
        departments[i][1] = result
        if(result > mostLikely[1]):
            mostLikely[0] = departments[i][0]
            mostLikely[1] = result

    departments = sorted(departments, key=lambda x : x[1], reverse=True)
    return mostLikely[0]

EDU_SYS = {
    "學士" : "學士班",
    "碩士" : "碩士班",
    "碩士專班" : "碩士在職專班",
    "博士" : "博士班"
}

FILTER = ["學制班別", "年級"]

EDU_SYS_DEPTS = {
    
}

STUDENT_HEAD = [
    "college_name",
    "dept_name",
    "education_system",
    "stu_no",
    "stu_name",
    "stu_year",
    "gender",
    "country_name"
]

class Row:
    
    def __init__(self, configure):
        # print(configure)
        # self.col = col
        self.configure = configure
        self.area = {}
    
    def append(self, student):
        pass

    


class Table4(Row):
    def __init__(self, configure, country_code):
        super().__init__(configure)
        self.country_code = country_code


    def append(self, values:pd.Series):
        try:
            country_code = self.country_code[self.country_code["國別"] == values["country_name"]].國別代碼.values[0]
        except:
            print(values)
            raise KeyError
        if not country_code in self.area:
            self.area[country_code] = {}
            self.area[country_code]["0"] = {}
            self.area[country_code]["1"] = {}
            self.area[country_code]["0"]["boy"] = []
            self.area[country_code]["0"]["girl"] = []

            self.area[country_code]["1"]["girl"] = []
            self.area[country_code]["1"]["boy"] = []
            

        if int(values["gender_code"]) == 1:
            if(country_code == 60 or country_code == 70):
                self.area[country_code]["1"]["boy"].append(values)
            else:
                self.area[country_code]["0"]["boy"].append(values)
        else:
            if(country_code == 60 or country_code == 70):
                self.area[country_code]["1"]["girl"].append(values)
            else:
                self.area[country_code]["0"]["girl"].append(values)
        # print(self.area)

    def output(self):
        outputData = []
        for key, item in self.area.items():
            for j in ("0", "1"):
                configure = self.configure.copy()
                configure["僑居地國別"] = key
                configure["僑生及港澳生學生人數_男"] = len(item[j]["boy"])
                configure["僑生及港澳生學生人數_女"] = len(item[j]["girl"])
                students = []
                for i in item[j]["boy"]:
                    students.append(list(i.get(STUDENT_HEAD).values))
                
                for i in item[j]["girl"]:
                    students.append(list(i.get(STUDENT_HEAD).values))
            
                outputData.append([list(configure.values), students])
            
        # print(test)
        return outputData
    

    def outputMainTable(self):
        outputData = []
        for key, item in self.area.items():
            for j in ("0", "1"):
                configure = self.configure.copy()
                configure["僑居地國別/地區/省市代碼"] = key
                configure["身分類別代碼"] = j
                configure["僑生及港澳生學生人數_男"] = len(item[j]["boy"])
                configure["僑生及港澳生學生人數_女"] = len(item[j]["girl"])
                
                if(len(item[j]["boy"]) != 0 or len(item[j]["girl"]) != 0):
                    outputData.append(configure)
        
        return outputData


# {
#     "學士班" : {
#         "數學系" : {
#             1 : Row()
#         }
#     }
# }

def constructDataStructure(config:pd.DataFrame, country_code):
    data = {}
    for i in range(len(config.values)):
        dept = config.loc[i]["單位"]
        edu_system = config.loc[i]["學制班別"]
        grade = config.loc[i]["年級"]

        if not edu_system in data:
            data[edu_system] = {}
        
        if not dept in data[edu_system]:
            data[edu_system][dept] = {}
        
        if not grade in data[edu_system][dept]:
            data[edu_system][dept][grade] = Table4(config.loc[i], country_code)

    # print(data.keys())

    return data



def cleanData(studentData:pd.DataFrame):
    studentData.education_system = studentData.education_system.str.strip(' ')
    studentData.country_name = studentData.country_name.str.strip(' ')
    studentData.dept_name = studentData.dept_name.str.replace(r' ','')
    studentData.temp_dept_name = studentData.dept_name.str.replace(r'\w士班', '')
    studentData.temp_dept_name = studentData.temp_dept_name.str.replace(r'應用數學', '')
    studentData.temp_dept_name = studentData.temp_dept_name.str.replace(r'碩士在職專班', '')
    studentData.temp_dept_name = studentData.temp_dept_name.str.replace(r' ', '')
    studentData.stu_name = studentData.stu_name.str.strip(r' ')
    studentData.college_name = studentData.college_name.str.strip(r' ')
    studentData.status = studentData.status.str.strip(r' ')
    studentData.fillna('', inplace=True)
    return studentData


def init(config):
    global EDU_SYS_DEPTS
    EDU_SYS_DEPTS["學士"] = list(set(config.query("學制班別 == '學士班'").單位))
    EDU_SYS_DEPTS["碩士"] = list(set(config.query("學制班別 == '碩士班'").單位))
    EDU_SYS_DEPTS["博士"] = list(set(config.query("學制班別 == '博士班'").單位))
    EDU_SYS_DEPTS["碩士專班"] = list(set(config.query("學制班別 == '碩士在職專班'").單位))
    
def departmentNameClassification(studentData, filename):
    studentData["temp_dept_name"] = np.full(studentData.shape[0], np.nan)
    studentData = cleanData(studentData)
    # Classifying
    data = {}
    for i in range(len(studentData.values)):
        dept_name = classify(studentData.loc[i]["temp_dept_name"], EDU_SYS_DEPTS[studentData.loc[i]["education_system"]])
        studentData["temp_dept_name"][i] = dept_name
        data[str(studentData.loc[i]["dept_name"])] = dept_name
    text = json.dumps(data, indent=4, ensure_ascii=False)
    with open(filename, "w") as file:
        file.write(text)
        file.close()
    return data

**First, read the table**

In [54]:
config = pd.read_excel("./第11003期_學4.僑生及港澳生學生(3月、10月填報).xls", dtype=str) # load table
studentData = pd.read_excel("./複本 校庫學生﹍for國際處﹍1100315-僑陸0316﹍刪除在職生.xls", sheet_name=1, dtype=str) # load data
country_code = pd.read_excel("./國別代碼對照表 (6).xls") # country_code

**Initialize the code**

In [66]:
studentData = cleanData(studentData)
config.fillna('', inplace=True)
dataStructure = constructDataStructure(config, country_code)
init(config)

# remove country_name == null
# remove status != "註冊(在學)"
studentData = studentData[studentData['country_name'] != '(null)']
studentData = studentData[studentData["status"] == "註冊(在學)"]
studentData = studentData.reset_index()

<ipython-input-23-bc49df3d322c>:191: FutureWarning: The default value of regex will change from True to False in a future version.
  studentData.temp_dept_name = studentData.dept_name.str.replace(r'\w士班', '')


**Open depts_map.json**

In [70]:
try:
    with open("dept_maps.json", "r") as file:
        depts_name = json.load(file)
except:
    print("ERROR: Can not open file.")
    filename = "dept_maps.json"
    depts_name = departmentNameClassification(studentData, filename)

In [71]:
# start classifing
for i in range(len(studentData.values)):
    edu_sys = EDU_SYS[studentData.loc[i]["education_system"]]
    stu_year = studentData.loc[i]["stu_year"]
    dept_name = depts_name[studentData["dept_name"][i]]
    try:
        dataStructure[edu_sys][ dept_name ][ stu_year ].append(studentData.loc[i])
    except:
        print(edu_sys, dept_name, stu_year)
        raise KeyError

In [76]:
# output classification result
statisticData = []
tableData = []
for i in dataStructure:
    for j in dataStructure[i]:
        for k in dataStructure[i][j]:
            statisticData += dataStructure[i][j][k].output()
            tableData += dataStructure[i][j][k].outputMainTable()
newdf = pd.DataFrame(tableData)
newdf.to_excel('output4-2_temp.xlsx')